# Automated ML

TODO: Import Dependencies. In the cell below, import all the dependencies that you will need to complete the project.

In [ ]:
from azureml.core import Workspace, Experiment
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException
from azureml.core.model import InferenceConfig
from azureml.core import Workspace, Dataset
from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.data.dataset_factory import TabularDatasetFactory
from azureml.core.webservice import AciWebservice
import os
from azureml.core import Environment
from azureml.core import ScriptRunConfig
from azureml.train.automl import AutoMLConfig
import pandas as pd
import joblib

## Dataset

### Overview
TODO: In this markdown cell, give an overview of the dataset you are using. Also mention the task you will be performing.


TODO: Get data. In the cell below, write code to access the data you will be using in this project. Remember that the dataset needs to be external.

In [ ]:
ws = Workspace.from_config()

# choose a name for experiment
experiment_name = 'automl'

experiment=Experiment(ws, experiment_name)

In [ ]:
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

#Create compute cluster if it not allready exitst
cpu_cluster_name = 'notebook147080'
try:
    my_compute_target = ComputeTarget(workspace=ws,name=cpu_cluster_name)
except ComputeTargetException:  
    compute_config = AmlCompute.provisioning_configuration(vm_size='Standard_D2_V2', max_nodes=4)
    my_compute_target = ComputeTarget.create(ws, cpu_cluster_name, compute_config)

my_compute_target.wait_for_completion(show_output=True)

In [ ]:
data_source = "https://raw.githubusercontent.com/Graflinger/Azure-ML-Engineer-Udacity-3/main/water_potability.csv"
ds = TabularDatasetFactory.from_delimited_files(path=data_source)

y_name = "Potability"

## AutoML Configuration

TODO: Explain why you chose the automl settings and cofiguration you used below.

Answer: The dataset and the neame of the target columns is provided, in order that the auto ml model got the needed data. Furthermore, it's defined that the task is a classification task and that the training should timeout after 30 minutes. The primary metric is accuracy and the training should stop if 99% accuracy is achieved.

In [ ]:
from azureml.train.automl import AutoMLConfig
import pandas as pd
from sklearn.model_selection import train_test_split

dataset = Dataset.get_by_name(workspace, name='water_quality')
y_name = "Potability"
# Set parameters for AutoMLConfig

automl_settings = {
    "enable_early_stopping" : True,
    "iteration_timeout_minutes": 5,
    "max_concurrent_iterations": 4,
    "max_cores_per_iteration": -1,
    "primary_metric": 'accuracy',
    "featurization": 'auto',
    "n_cross_validations":5
}

automl_config = AutoMLConfig(experiment_timeout_minutes=30,
                             task = 'classification',
                             debug_log = 'automl_errors.log',
                             compute_target=my_compute_target,
                             experiment_exit_score = 0.99,
                             enable_onnx_compatible_models=True,
                             training_data = ds,
                             label_column_name = y_name,
                             **automl_settings
                            )


In [ ]:
run = experiment.submit(automl_config)

## Run Details

OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?

TODO: In the cell below, use the `RunDetails` widget to show the different experiments.

In [ ]:
RunDetails(run).show()
run.wait_for_completion(show_output=True)

## Best Model

TODO: In the cell below, get the best model from the automl experiments and display all the properties of the model.



In [ ]:
# Retrieve and save your best automl model.

best_run, fitted_model = run.get_output()
print(fitted_model.steps)

model_name = best_run.properties['model_name']
description = 'AutoML'
tags = None

model = run.register_model(model_name = model_name, 
                                  description = description, 
                                  tags = tags)

In [ ]:
print(fitted_model)

In [ ]:
#TODO: Save the best model
joblib.dump(best_run, 'best_model_automl.pkl')

## Model Deployment

Remember you have to deploy only one of the two models you trained.. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

TODO: In the cell below, send a request to the web service you deployed to test it.

TODO: In the cell below, print the logs of the web service and delete the service